**Falta implementar de manera correcta GPT-OSS**

In [1]:
import torch
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(dev)

cpu


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, GptOssConfig
from peft import LoraConfig, TaskType

# OBS: Para cuantizar a gpt-oss, es necesario usar MXFP4 en vez de BitsAndBytes. ¿Por qué? Porque sam altaman y openai son hitler.

model_id = "openai/gpt-oss-20b"

#quant_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = torch.bfloat16)
gen_config = GenerationConfig.from_pretrained("openai/gpt-oss-20b")

cfg = GptOssConfig.from_pretrained(model_id)
print(cfg.quantization_config)

oss_tokenizer = AutoTokenizer.from_pretrained(model_id)
oss_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = cfg.quantization_config)
print(oss_tokenizer)
print("======================================")
print("======================================")
print(oss_model)

{'modules_to_not_convert': ['model.layers.*.self_attn', 'model.layers.*.mlp.router', 'model.embed_tokens', 'lm_head'], 'quant_method': 'mxfp4'}


ValueError: The model is quantized with Mxfp4Config but you are passing a dict config. Please make sure to pass the same quantization config class to `from_pretrained` with different loading attributes.

In [ ]:
# Otra alternativa
from transformers import Mxfp4Config

quantization_config2 = Mxfp4Config(dequantize=True)
model_kwargs = dict(
    attn_implementation = "eager",
    torch_dtype = torch.bfloat16, 
    quantization_config = quantization_config2,
    use_cache = False,
    device_map = dev,
)
model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)

ValueError: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires `accelerate`. You can install it with `pip install accelerate`